- Bu çalışmada yapılan ön işleme adımlarının %99'u, tez ve bilimsel araştırmaların referans alınması ile yapılmıştır.
- Önce veri seti ve değişkenler tanıtıldı, literatürdeki thresholdlar belirtildi.
- Veri setindeki tüm 0'ların alınamayan değerleri yani NA'leri temsil ettiği literatür araştırmalarından bilinmektedir.
- Önce 0'lara NA atandı, sonra bu NA'ler na olmayan df'teki target kırılımlarına göre dolduruldu.
- Base model kuruldu. (XGB: 0.889)
- Alınan ilk feature importance ile 4 yeni değişken oluşturuldu, Age değişkeni veri setinden atıldı.Model sonucu: XGB: 0.897180
- LOF ile ilk 50 değer drop edildi, Light GBM: 0.901154
- Verinin son haline lojistic regresyon uygulanarak overfit olup olmadığının anlaşılması hedeflendmiştir. 
- Overfit olmadığı ve recall değerinin düşük olduğu gözlemlenmiştir. Bu durumu düzelmeye yönelik bir eylemde bulunulmamıştır.
- Not: Robust Scaler denenerek sonuçların daha kötü olduğu anlaşılmış olup çalışmadan kaldırılmıştır.

# INFORMATION ABOUT THE DATASET


    
# Veri seti üzerinde yapılan araştırmalara göre,

- Veri seti ABD'deki Ulusal Diyabet-Sindirim-Böbrek Hastalıkları Enstitüleri'nde tutulan büyük veri setinin parçasıdır.
- ABD'deki Arizona Eyaleti'nin en büyük 5. şehri olan Phoenix şehrinde yaşayan 21 yaş ve üzerinde olan Pima Indian kadınları
üzerinde yapılan diyabet araştırması için kullanılan verilerdir.(Bazı kaynaklarda Hintli ya da bazılarında Kızılderili olarak geçiyorlar.)
- 768 gözlem, 8 numerik bağımsız değişkenden oluşmaktadır. Hedef değişken "outcome" olarak belirtilmiş olup 1 diyabet test sonucunun pozitif oluşunu, 0 ise negatif oluşunu belirtmektedir. (268:1, 500:0)

# Değişkenler:

- **Pregnancies**: Hamilelik Sayısı
- **Glucose**: OGTT: Oral glukoz tolerans testi gebelere yapılır. İlk defa gebelikte başlayan veya teşhis edilen glukoz intoleransı ölçüm testi.Gestasyonel diyabet tanısı konur. 
    - 2 saatlik yapılan OGTT'yi temsil etmektedir.(tek aşamalı)
    - 140 mg/dL >= : diyabet riskinin arttığı durumdur.
    -  
- **BloodPressure**: DBP Diastolik kan basıncını mmHg cinsinden temsil etmektedir.
    
    - <85 :2 yılda bir kontrol edilmeli
    - 85-89 : Senede bir
    - 90-99 : 2 ay içinde 
    - 100-109 : 1 ay içinde
    - 110 > : 1 hafta içinde ya da hemen

- **SkinThickness**: TSFT Triceps skinfold thickness (mm).        
        
- **Insulin**: Two-hour serum insulin (μU/mL) (2HSI). Insülin testi(tokluk 2 saat)
    - 120 altı normal- 140-199 gizli şeker- 200 ve üzeri diyabet

- **BMI** :Body Mass Index : kilo/ boy^2
    - <18.5 zayıf
    - 18.5-24.9 sağlıklı
    - 25-29.9 kilolu
    - 30-40 şişman
    - 40.1-60 aşırı şişman
        
- **DiabetesPedigreeFunction** : Soyağacı  
- **Age** : Yaş ( diyabet ise yaş median değeri 36, değilse 25)
- **Outcome**

# DİKKAT

- **Misingler değerlerin olmayışını temsil etmektedir.** 
    - (Referans: https://www.sciencedirect.com/science/article/pii/S2352914816300016#bib65 (3.7. madde))
    
# References:
- https://www.sciencedirect.com/science/article/pii/S2352914816300016#bib65 (3.7. madde)
- https://tr.ulfsciences.com/native-america-s-alleles-44932
- https://www.foodandnutritionjournal.org/volume7number2/significance-of-health-related-predictors-of-diabetes-in-pima-indians-women/
- https://www.novonordisk.com.tr/hastalara-ozel/diyabet/diyabet-tanisi-ve-tipleri.html
- https://tkd.org.tr/kilavuz/k03/3_18530.htm?wbnum=1103
- https://aysetugbasengel.com/kan-sekeri-degerleri-ne-olmali-aclik-tokluk-kan-sekeri-olcumu/#Tokluk_Kan_Sekeri_Testi_Nedir,_Nasil_Yapilir
- http://www.taylankumeli.com/hesaplama/bmi-hesapla/l

In [ ]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd 
import seaborn as sns
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import RobustScaler

import warnings
warnings.simplefilter(action = "ignore") 


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# READ THE DATA

diabetes= pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
df=diabetes.copy()

# DATA UNDERSTANDING

In [ ]:
# AT FIRST GLANCE

def info_(dataframe):
    print("DF SHAPE:",dataframe.shape, "\n")
    print("OUTCOME 1/ALL DF RATIO:",len(df[df["Outcome"]==1])/len(df), "\n")
    print("OUTCOME 0/ALL DF RATIO:",len(df[df["Outcome"]==0])/len(df), "\n")
    print("NUMBER OF NULL VALUES:", df.isnull().any(axis=1).sum(), "\n")
    print(dataframe.head(3), "\n")
    object_list=[col for col in dataframe.columns if dataframe[col].dtypes=="O"]
    numeric_list=[col for col in dataframe.columns if dataframe[col].dtypes!="O"]
    print("ALL COLUMNS:", df.columns, "\n")
    print("OBJECT COLUMNS:",object_list,"\n")
    print("NUMERIC COLUMNS",numeric_list,"\n")
    print("CORRELATIONS",sns.heatmap(df.corr()));
    print(sns.pairplot(df));
    return object_list, numeric_list

In [ ]:
obj_list, num_list=info_(df)

# DATA PREPROCESSING

In [ ]:
# 0'ların NA değerler olduğunu yukarıda verilen referanslardan biliyorduk.
# Bu kısımda önce Df'in 0'larının atılmış halini bulacağım. 
#Sonra NA değerlerine, bu sıfırsız df'e yaptığım kırılımla atama yapacağım.

In [ ]:
df1=df.copy()
df1=df1.loc[:,"Pregnancies": "Age"].replace(0, np.nan).dropna()

In [ ]:
for i in df1.columns:
    if i!="Outcome":
        print(i.upper(),"\n","for diabet:", df1.loc[df["Outcome"]==1, i].median(),"\n","for non-diabet:" , df1.loc[df["Outcome"]==0, i].median())

# TÜM SÜTUNLARDAKİ 0 DEĞERLERİNE KIRILIMLA ATAMA

In [ ]:
for i in df.columns:
    for k in range(len(df)):
        if i!="Outcome":
            if df.loc[k,i]==0: #eğer değer 0'a eşitse
                if df.loc[k,"Outcome"]==1:   #bu değerin outcome'ı 1'se
                     df.loc[k,i]=df1.loc[df["Outcome"]==1, i].median()# na değer olmayan df'teki outcome'ı 1 olanların i medyanını ata.
                elif df.loc[k,"Outcome"]==0:
                     df.loc[k,i]=df1.loc[df["Outcome"]==0, i].median()# na değer olmayan df'teki outcome'ı 0 olanların i medyanını ata.


# BASE MODEL : XGB: 0.889371

In [ ]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis = 1)
log_model = LogisticRegression().fit(X,y)
y_pred = log_model.predict(X)
print(accuracy_score(y, y_pred))
print(classification_report(y, y_pred))

In [ ]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis = 1)



models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('XGB', GradientBoostingClassifier()))
models.append(("LightGBM", LGBMClassifier()))

# evaluate each model in turn
results = []
names = []



for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 123456)
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        

# Base Model: Feature Importance 

In [ ]:
Importance = pd.DataFrame({'Importance':GradientBoostingClassifier().fit(X, y).feature_importances_*100}, 
                          index = X.columns)

Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', figsize=(8,6))

plt.xlabel('Variable Importance')
plt.gca().legend_ = None

# NEW FEATURES : XGB: 0.897180

In [ ]:
df2=df.copy()
df2["Insulin/Age"]=df2["Insulin"]/df2["Age"]
df2["BMI/Age"]=df2["BMI"]/df2["Age"]
df2["Pregnancies/Age"]=df2["Pregnancies"]/df2["Age"]
df2["Ins*Glu"]=df2["Insulin"]* df2["Glucose"]

df2=df2.drop(["Age"], axis=1)

In [ ]:
#ÇEŞİTLİ FEATURE DENEMELERİ
#df2["BMI*Skin"]=df2["BMI"]* df2["SkinThickness"]
#df2=df2.drop([DiabetesPedigreeFunction","BloodPressure","SkinThickness"], axis=1)
#df2=df2.drop(["BMI","SkinThickness"], axis=1)

In [ ]:
y = df2["Outcome"]
X = df2.drop(["Outcome"], axis = 1)

models = []
models.append(('XGB', GradientBoostingClassifier()))
models.append(("LightGBM", LGBMClassifier()))

# evaluate each model in turn
results = []
names = []

for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 123456)
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        

# LOF: LightGBM: 0.901154

In [ ]:
def lof(dataframe, n,c, numlist, plot_xlim,thr):
    clf=LocalOutlierFactor(n_neighbors=n, contamination=c)
    clf.fit_predict(dataframe[numlist])
    df_scores= clf.negative_outlier_factor_
    threshold=np.sort(df_scores)[thr]
    print(np.sort(df_scores)[0:50])
    pd.DataFrame(np.sort(df_scores)).plot(stacked=True,xlim=[0,plot_xlim], style=".-");
    plt.show()
    return df_scores,threshold 

In [ ]:
cols=[col for col in df2.columns if col!="Outcome"]
scores, threshold =lof(df2, 20, 0.1, cols, 100,50)

In [ ]:
df3=df2.drop(df2[scores< threshold].index, axis=0).reset_index(drop=True)

In [ ]:
y = df3["Outcome"]
X = df3.drop(["Outcome"], axis = 1)

models = []
models.append(('XGB', GradientBoostingClassifier()))
models.append(("LightGBM", LGBMClassifier()))

# evaluate each model in turn
results = []
names = []

for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 123456)
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        

In [ ]:
# Overfit var mı?
y = df3["Outcome"]
X = df3.drop(["Outcome"], axis = 1)
log_model = LogisticRegression().fit(X,y)
y_pred = log_model.predict(X)
print(accuracy_score(y, y_pred))
print(classification_report(y, y_pred))